### MQTT Client to control neopixel light

### 要用到paho來傳mqtt message

- install: `pip install paho-mqtt`


- 兩色交替閃爍

In [ ]:
#!/usr/bin/python
# -*- coding: utf-8 -*-
import paho.mqtt.client as mqtt  #import the client1
import time

import time
from struct import *
import random
import json

def on_connect(client, userdata, flags, rc):
    m="Connected flags"+str(flags)+", result code "+str(rc)+", client_id  "+str(client)
    print(m)

# some online free broker:
#   iot.eclipse.org
#   test.mosquitto.org
#   broker.hivemq.com
broker_address = "broker.hivemq.com"
TOPIC_BASE = 'pochang/iot'
topic_light = TOPIC_BASE+"/light"
topic_neopixel = TOPIC_BASE+"/neopixel"

client1 = mqtt.Client()    #create new instance
client1.on_connect = on_connect        #attach function to callback

time.sleep(1)
client1.connect(broker_address, 1883, 60)      #connect to broker

#client1.loop_forever()
# 有自己的while loop，所以call loop_start()，不用loop_forever
client1.loop_start()    #start the loop
time.sleep(2)
print("loop start")

while True:
    
    neo_cmd = {'light': [[0,10,10]]*8, 'delay':0.5}
    neo_cmd = json.dumps(neo_cmd)
    print('cmd: ', neo_cmd)
    client1.publish(topic_neopixel, neo_cmd)
    time.sleep(2)

    neo_cmd = {'light': [[10,0,0]]*8, 'delay':0.5}
    neo_cmd = json.dumps(neo_cmd)
    print('cmd: ', neo_cmd)
    client1.publish(topic_neopixel, neo_cmd)
    time.sleep(2)
    

## neo-light flow~

In [ ]:
#!/usr/bin/python
# -*- coding: utf-8 -*-
import paho.mqtt.client as mqtt  #import the client1
import time

import time
from struct import *
import random
import json

def on_connect(client, userdata, flags, rc):
    m="Connected flags"+str(flags)+", result code "+str(rc)+", client_id  "+str(client)
    print(m)

# some online free broker:
#   iot.eclipse.org
#   test.mosquitto.org
#   broker.hivemq.com
broker_address = "broker.hivemq.com"
TOPIC_BASE = 'pochang/iot'
topic_light = TOPIC_BASE+"/light"
topic_neopixel = TOPIC_BASE+"/neopixel"

client1 = mqtt.Client()    #create new instance
client1.on_connect = on_connect        #attach function to callback

time.sleep(1)
client1.connect(broker_address, 1883, 60)      #connect to broker

#client1.loop_forever()
# 有自己的while loop，所以call loop_start()，不用loop_forever
client1.loop_start()    #start the loop
time.sleep(2)
print("loop start")
idx = 0
while True:
    idx+=1
    
    neo_light = [ [0,0,0] ]*8
    v = 5
    for i in range(1, 5):
        neo_light[(idx+i)%8] = [v*i, 0, v*i]
    neo_cmd = {'light': neo_light, 'delay':0}
    neo_cmd = json.dumps(neo_cmd)
    print('cmd: ', neo_cmd)
    client1.publish(topic_neopixel, neo_cmd)
    time.sleep(0.5)


In [ ]:
neo_cmd = {'neopixel': [[10,0,0]]*8, 'delay':0.5}
neo_cmd = json.dumps(neo_cmd)
print('cmd: ', neo_cmd)


----


### live demo

- 以先架好的rpi3控制PLC為例


In [ ]:
import paho.mqtt.client as mqtt  #import the client1
import time

import serial
import modbus_tk
import modbus_tk.defines as cst
import modbus_tk.modbus_rtu as modbus_rtu
import time

mbComPort = 'COM5'
baudrate = 9600
databit = 8
parity = 'N'
stopbit = 1
mbTimeout = 100 # ms

def on_connect(client, userdata, flags, rc):
    m="Connected flags"+str(flags)+", result code "+str(rc)+", client_id  "+str(client)
    print(m)

    # first value OFF
    print('set light off!')
    control_light(0)
    client1.publish(topic,0)    

def on_message(client1, userdata, message):
    print("message received  "  ,str(message.payload.decode("utf-8")), message.topic, message.qos, message.retain)
    if message.topic == topic:
        my_message = str(message.payload.decode("utf-8"))
        print("set light: ", my_message)
        if my_message=='1' or my_message==1:
            control_light(1)
        else:
            control_light(0)

def on_log(client, userdata, level, buf):
    print("log: ",buf)

def control_light(value):
    mb_port = serial.Serial(port=mbComPort, baudrate=baudrate, bytesize=databit, parity=parity, stopbits=stopbit)
    master = modbus_rtu.RtuMaster(mb_port)
    master.set_timeout(mbTimeout/1000.0)

    mbId = 1
    addr = 2 #base0

    try:
        #-- FC5: write multi-coils
        rr = master.execute(mbId, cst.WRITE_SINGLE_COIL, addr, output_value=value)
        print("Write(addr, value)=%s" %(str(rr)))

    except Exception as e:#except Exception, e:
        print("modbus test Error: " + str(e))

    master._do_close()


# some online free broker:
#   iot.eclipse.org
#   test.mosquitto.org
#   broker.hivemq.com
broker_address="iot.eclipse.org"
topic = "malo-iot/light"
client1 = mqtt.Client()    #create new instance
client1.on_connect = on_connect        #attach function to callback
client1.on_message = on_message        #attach function to callback
#client1.on_log=on_log

time.sleep(1)
client1.connect("iot.eclipse.org", 1883, 60)      #connect to broker
#client1.loop_start()    #start the loop
client1.subscribe(topic)

client1.loop_forever()

----

## Web API

適合非即時性的應用

----


### 自己做restful API + PostgreSQL

### firebase
    
- 上傳時一個Json帶入所有點位

- 顯示資料時可一次一個點位處理

- 改寫輸出值時一次一個點位更是方便

### ThingSpeak

- 上傳資料自動畫成趨勢圖


----

### 運用Firebase收集資料

[demo](Firebase.ipynb): 要新增一個firebase的點，做台中.py課程臨時上傳用

- firebase 套件安裝
    - [link](https://pypi.org/project/python-firebase/)
    - pip install python-firebase
    
- 不用套件的用法

----


In [ ]:
tag_name = '/tag_1'

In [ ]:
from firebase import firebase
import time

my_firebase = firebase.FirebaseApplication('https://fire-test-c46d1.firebaseio.com', None)

my_time = time.strftime("%Y-%m-%d %H:%M:%S")
my_data = {'data': {'T': 25, 'H': 65}, 'time': my_time}
result = my_firebase.post(tag_name+'/log', my_data)
print('post result=', result)

result = my_firebase.put('', '/'+tag_name+'/rt', my_data)
print('put result=', result)


- 看一下：https://fire-test-c46d1.firebaseio.com/.json

In [ ]:
from firebase import firebase
import time

my_firebase = firebase.FirebaseApplication('https://fire-test-c46d1.firebaseio.com', None)

result = my_firebase.get(tag_name, None)
result

----

### 運用 ThingSpeak收集資料

demo

----

In [ ]:
# push data

import requests

# field1: T
# field2: H
url = 'https://api.thingspeak.com/update'
api_key = 'RD1VBDRERV09LPV4'
field1 = 24.58
field2 = 63.11
data = {'api_key': api_key, 'field1':field1, 'field2':field2}
r = requests.get(url, params=data)
r

In [ ]:
r.text

In [ ]:
# continue push data

my_h = [65.46, 65.75, 62.49, 65.4, 65.89, 63.11]
my_t = [25.05, 24.72, 25.06, 24.66, 24.99, 24.58]

In [ ]:
len(my_t)

In [ ]:
import requests
import time

for i in range(len(my_t)):
    
    print('---- %s ----' %(i))
    print('H=%s %%, T=%s degree' %(my_h[i], my_t[i]))
    
    # push data
    # field1: T
    # field2: H
    url = 'https://api.thingspeak.com/update'
    api_key = 'RD1VBDRERV09LPV4'
    field1 = my_t[i]
    field2 = my_h[i]
    data = {'api_key': api_key, 'field1':field1, 'field2':field2}
    r = requests.get(url, params=data)
    print(r)
    
    time.sleep(15)
    

----

## The End !?

----


### 接著就是UI, UX工程師，資料分析工程師的事了!?
    
資料分析不只是資料分析工程師的事


### 未來各種AI功能可能都會變成Plug-in的物件

舉例來說：

- 拼裝車、修車的技士

- 電機儀器的維修工程師

- 金屬加工機的電控工程師

- 未來可能會出現AI調校工程師


----

## Q&A

----